In [1]:
import duckdb
from pathlib import Path

In [2]:
connection = duckdb.connect(':memory:')

In [3]:
create_table_query = f"""
CREATE OR REPLACE TABLE new(token_1 UINTEGER, count UINTEGER, log_likelihood DOUBLE DEFAULT 0, PRIMARY KEY(token_1))
"""
connection.execute(create_table_query)

In [4]:
new_table = connection.sql('SELECT * FROM new')
print(len(new_table))
new_table

0


┌─────────┬────────┬────────────────┐
│ token_1 │ count  │ log_likelihood │
│ uint32  │ uint32 │     double     │
├───────────────────────────────────┤
│              0 rows               │
└───────────────────────────────────┘

In [5]:
parquet_file = Path('data/1/count_table_0-999999.parquet')
parquet_file

WindowsPath('data/1/count_table_0-999999.parquet')

In [6]:
table_to_insert = connection.from_parquet(str(parquet_file))
print(len(table_to_insert))
table_to_insert

21237


┌───────────────┬────────┐
│    token_1    │ count  │
│     int64     │ int64  │
├───────────────┼────────┤
│          2788 │   1097 │
│          1010 │ 634525 │
│          2002 │ 249401 │
│          2052 │  44729 │
│          2022 │  55340 │
│         13311 │    326 │
│          2105 │  20016 │
│          1996 │ 477275 │
│          2542 │   2668 │
│          2282 │  12122 │
│            ·  │     ·  │
│            ·  │     ·  │
│            ·  │     ·  │
│         17114 │    197 │
│         22119 │    138 │
│          3505 │    317 │
│         14688 │    284 │
│         18915 │    130 │
│          9306 │     89 │
│          6341 │     18 │
│         10920 │     89 │
│         25937 │     38 │
│         12822 │     43 │
├───────────────┴────────┤
│ ? rows       2 columns │
└────────────────────────┘

In [7]:
insert_query1 = """
INSERT INTO new 
    SELECT token_1, count, 0 FROM table_to_insert
    ON CONFLICT (token_1) DO UPDATE
        SET count = count + excluded.count;
"""
connection.execute(insert_query1)

In [8]:
new_table = connection.sql('SELECT * FROM new')
print(len(new_table))
new_table

21237


┌─────────┬────────┬────────────────┐
│ token_1 │ count  │ log_likelihood │
│ uint32  │ uint32 │     double     │
├─────────┼────────┼────────────────┤
│    2788 │   1097 │            0.0 │
│    1010 │ 634525 │            0.0 │
│    2002 │ 249401 │            0.0 │
│    2052 │  44729 │            0.0 │
│    2022 │  55340 │            0.0 │
│   13311 │    326 │            0.0 │
│    2105 │  20016 │            0.0 │
│    1996 │ 477275 │            0.0 │
│    2542 │   2668 │            0.0 │
│    2282 │  12122 │            0.0 │
│      ·  │     ·  │             ·  │
│      ·  │     ·  │             ·  │
│      ·  │     ·  │             ·  │
│   17114 │    197 │            0.0 │
│   22119 │    138 │            0.0 │
│    3505 │    317 │            0.0 │
│   14688 │    284 │            0.0 │
│   18915 │    130 │            0.0 │
│    9306 │     89 │            0.0 │
│    6341 │     18 │            0.0 │
│   10920 │     89 │            0.0 │
│   25937 │     38 │            0.0 │
│   12822 │ 

In [9]:
parquet_file2 = Path('data/1/count_table_1000000-1999999.parquet')
table_to_insert2 = connection.from_parquet(str(parquet_file2))
print(len(table_to_insert2))
table_to_insert2

20761


┌───────────────┬────────┐
│    token_1    │ count  │
│     int64     │ int64  │
├───────────────┼────────┤
│          1037 │ 208277 │
│          3730 │   3020 │
│          7324 │    644 │
│          2006 │  78829 │
│          1996 │ 400332 │
│          2341 │  10425 │
│          2081 │  13989 │
│          2033 │ 113129 │
│          5376 │    584 │
│          1012 │ 945786 │
│            ·  │      · │
│            ·  │      · │
│            ·  │      · │
│         17959 │     55 │
│         17364 │     64 │
│         25289 │     20 │
│         16153 │    235 │
│         20699 │     27 │
│         27801 │    121 │
│         28625 │      4 │
│          7712 │     36 │
│         17284 │     16 │
│         17813 │     32 │
├───────────────┴────────┤
│ ? rows       2 columns │
└────────────────────────┘

In [10]:
insert_query2 = f"""
INSERT INTO new 
    SELECT token_1, count, 0 FROM table_to_insert2
    ON CONFLICT (token_1) DO UPDATE 
        SET count = count + excluded.count;
"""
connection.execute(insert_query2)

In [11]:
new_table = connection.sql('SELECT * FROM new')
print(len(new_table))
new_table

22548


┌─────────┬─────────┬────────────────┐
│ token_1 │  count  │ log_likelihood │
│ uint32  │ uint32  │     double     │
├─────────┼─────────┼────────────────┤
│    2788 │    2277 │            0.0 │
│    1010 │ 1241213 │            0.0 │
│    2002 │  473866 │            0.0 │
│    2052 │   79586 │            0.0 │
│    2022 │  104493 │            0.0 │
│   13311 │     527 │            0.0 │
│    2105 │   38128 │            0.0 │
│    1996 │  877607 │            0.0 │
│    2542 │    4746 │            0.0 │
│    2282 │   22159 │            0.0 │
│      ·  │      ·  │             ·  │
│      ·  │      ·  │             ·  │
│      ·  │      ·  │             ·  │
│   17114 │     217 │            0.0 │
│   22119 │     214 │            0.0 │
│    3505 │     768 │            0.0 │
│   14688 │     583 │            0.0 │
│   18915 │     188 │            0.0 │
│    9306 │     118 │            0.0 │
│    6341 │      64 │            0.0 │
│   10920 │     156 │            0.0 │
│   25937 │      98 │    